# **1. Mô tả ứng dụng** 

*   **Ứng dụng mà nhóm chọn:** huấn luyện Convolutional Neural Network để phân lớp ảnh bằng Stochastic Gradient Descent.
   *   **Input:** 1 bức ảnh số viết tay có kích thước 28x28.

   *   **Output:** số được viết trong bức ảnh.

   *   **Ý nghĩa của ứng dụng trong thực tế:** giúp máy tính có thể tự nhận biết trong hình là số mấy, từ đó tăng tốc độ cho các tác vụ liên quan.

*   **Ứng dụng này có cần phải tăng tốc không:** có, vì ứng dụng huấn luyện Convolutional Neural Network có tiềm năng song song hóa vì trong đó có phép convolution, phép nhân ma trận.

# **2. Cài đặt tuần tự**

*   **Thiết kế:**
*   **Đánh giá:**


# **3. Cài đặt song song (trên GPU)**

*   **Phân tích:**
*   **Thiết kế:**
*   **Đánh giá:**

# **4. Cài đặt song song (trên GPU) + tối ưu hóa**

# **5. Nhìn lại quá trình làm đồ án**

# **6. Tài liệu tham khảo**

https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist

https://victorzhou.com/blog/intro-to-cnns-part-1/

https://victorzhou.com/blog/intro-to-cnns-part-2/